# Hipotezy statystyczne i założenia eksperymentu

## Kontekst badania

Celem badania jest porównanie modeli BERT uczonych w podejściu jednozadaniowym
oraz wielozadaniowym w zadaniach klasyfikacji:
- gatunków (genre),
- słów kluczowych (keywords).

Modele zostały ocenione z wykorzystaniem 5-krotnej walidacji krzyżowej.

In [55]:
import pandas as pd
from scipy.stats import wilcoxon
from statsmodels.stats.multitest import multipletests
import numpy as np

In [56]:
import pandas as pd

single_genres = pd.read_csv("singletask_classifier/results/cross_val_genres_metrics.csv")
single_keywords = pd.read_csv("singletask_classifier/results/cross_val_keywords_metrics.csv")
multitask = pd.read_csv("multitask_classifier/results/cross_val_multitask_metrics.csv")

# sanity check
len(single_genres),len(multitask),len(single_keywords)


(5, 5, 5)

In [57]:
GENRE_METRICS = [
    "genre_f1_macro",
    "genre_f1_micro",
    "genre_precision_micro",
    "genre_recall_micro",
    "genre_jaccard_samples",
    "genre_hamming_accuracy",
]

KEYWORD_METRICS = [
    "keyword_f1_macro",
    "keyword_f1_micro",
    "keyword_precision_micro",
    "keyword_recall_micro",
    "keyword_jaccard_samples",
    "keyword_hamming_accuracy",
]


## Hipotezy statystyczne 

**Hipoteza zerowa (H₀):**

Model wielozadaniowy nie różni się istotnie statystycznie od modelu jednozadaniowego w zadaniu klasyfikacji gatunków.

**Hipoteza alternatywna (H₁):**

Model wielozadaniowy różni się istotnie statystycznie od modelu jednozadaniowego w zadaniu klasyfikacji gatunków.

## Testy statystyczne

Do weryfikacji hipotez zastosowano:

- **Wilcoxon signed-rank test** (test rang znakowanych Wilcoxona),
- test **dwustronny**,
- poziom istotności: \( \alpha = 0.05 \).


In [62]:
genre_results = []

for metric in GENRE_METRICS:
    x = multitask[metric]
    y = single_genres[metric]

    stat, p = wilcoxon(
        x,
        y,
        alternative="two-sided"
    )

    genre_results.append({
        "metric": metric,
        "mean_multitask": x.mean(),
        "mean_single": y.mean(),
        "difference": x.mean() - y.mean(),
        "wilcoxon_stat": stat,
        "p_value": p,
    })

genre_df = pd.DataFrame(genre_results)
genre_df

,metric,mean_multitask,mean_single,difference,wilcoxon_stat,p_value
0,genre_f1_macro,0.575190,0.584318,-0.009128,0.0,0.0625
1,genre_f1_micro,0.624165,0.631989,-0.007824,1.0,0.1250
2,genre_precision_micro,0.498176,0.514785,-0.016609,0.0,0.0625
3,genre_recall_micro,0.835701,0.818356,0.017345,0.0,0.0625
4,genre_jaccard_samples,0.479612,0.493979,-0.014366,0.0,0.0625
5,genre_hamming_accuracy,0.829804,0.840146,-0.010342,0.0,0.0625


In [63]:
keyword_results = []

for metric in KEYWORD_METRICS:
    x = multitask[metric]
    y = single_keywords[metric]

    stat, p = wilcoxon(
        x,
        y,
        alternative="two-sided"
    )

    keyword_results.append({
        "metric": metric,
        "mean_multitask": x.mean(),
        "mean_single": y.mean(),
        "difference": x.mean() - y.mean(),
        "wilcoxon_stat": stat,
        "p_value": p,
    })

keyword_df = pd.DataFrame(keyword_results)
keyword_df

,metric,mean_multitask,mean_single,difference,wilcoxon_stat,p_value
0,keyword_f1_macro,0.208086,0.188774,0.019312,0.0,0.0625
1,keyword_f1_micro,0.206611,0.184661,0.021950,0.0,0.0625
2,keyword_precision_micro,0.124952,0.108290,0.016662,0.0,0.0625
3,keyword_recall_micro,0.596698,0.627827,-0.031129,0.0,0.0625
4,keyword_jaccard_samples,0.125371,0.107103,0.018268,0.0,0.0625
5,keyword_hamming_accuracy,0.835864,0.801366,0.034499,0.0,0.0625


## Podsumowanie
Wszystkie p-value = 0.0625 – minimalne możliwe przy 5 foldach. 
H₀ nie jest odrzucone, test nie wykazuje istotnej różnicy, choć średnie różnice sugerują przewagę modelu multitask w większości metryk.